<a href="https://colab.research.google.com/github/mfilipelino/pynotebook/blob/main/SummaryStatsAndWindowFunctions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark==3.1.1 py4j==0.10.9 

     |████████████████████████████████| 212.3MB 76kB/s 
     |████████████████████████████████| 204kB 41.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=6180165c48147aaf3ed7dd95f2e786ac0bcd6064ecbb3e0c56c2f27adca1b361
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window


spark = SparkSession.builder\
        .master("local[*]")\
        .appName('pyspark_notebook')\
        .getOrCreate()

In [1]:
df_ecommerce = spark.read.format('csv') \
  .option('header', 'true') \
  .option('inferSchema', 'true') \
  .load('drive/MyDrive/datasets/summer.csv')

df_ecommerce.createOrReplaceTempView('Summer')

NameError: ignored

In [ ]:
df_ecommerce.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [ ]:
query = """
SELECT
  Country, COUNT(CustomerID), round(SUM(UnitPrice * Quantity), 2) as total
FROM Ecommerce
GROUP BY Country
ORDER BY total
"""
spark.sql(query).show()


+--------------------+-----------------+--------+
|             Country|count(CustomerID)|   total|
+--------------------+-----------------+--------+
|        Saudi Arabia|               10|  131.17|
|             Bahrain|               17|   548.4|
|      Czech Republic|               30|  707.72|
|                 RSA|               58| 1002.31|
|              Brazil|               32|  1143.6|
|  European Community|               61| 1291.75|
|           Lithuania|               35| 1661.06|
|             Lebanon|               45| 1693.88|
|                 USA|              291| 1730.92|
|United Arab Emirates|               68| 1902.28|
|               Malta|              127| 2505.47|
|              Canada|              151| 3666.38|
|             Iceland|              182|  4310.0|
|              Greece|              146| 4710.52|
|         Unspecified|              244| 4749.79|
|              Poland|              341| 7213.14|
|              Israel|              250| 7907.82|


In [ ]:
query = """
SELECT 
  Country, Quantity, UnitPrice, ROUND((Quantity * UnitPrice), 2) as TotalPrice
FROM Ecommerce
WHERE Country = 'Brazil'
"""
spark.sql(query).show(100)

+-------+--------+---------+----------+
|Country|Quantity|UnitPrice|TotalPrice|
+-------+--------+---------+----------+
| Brazil|      16|    10.95|     175.2|
| Brazil|      24|     2.55|      61.2|
| Brazil|      24|     2.55|      61.2|
| Brazil|      24|     2.55|      61.2|
| Brazil|      10|     6.75|      67.5|
| Brazil|      24|     3.39|     81.36|
| Brazil|      24|     1.95|      46.8|
| Brazil|      10|     1.65|      16.5|
| Brazil|      12|     1.95|      23.4|
| Brazil|      10|     1.65|      16.5|
| Brazil|       3|     5.95|     17.85|
| Brazil|       3|     5.95|     17.85|
| Brazil|       3|     5.95|     17.85|
| Brazil|       3|     5.95|     17.85|
| Brazil|      12|     2.08|     24.96|
| Brazil|      12|      2.1|      25.2|
| Brazil|      10|     4.95|      49.5|
| Brazil|      24|     1.25|      30.0|
| Brazil|      24|     3.45|      82.8|
| Brazil|       6|     2.89|     17.34|
| Brazil|       6|     2.89|     17.34|
| Brazil|      12|     1.25|      15.0|
